In [ ]:
# Importamos las librerias que vamos a usar
import requests
from urllib.error import HTTPError
from urllib.error import URLError
from bs4 import BeautifulSoup
from bs4.dammit import EncodingDetector
import pandas as pd
import numpy as np
import datetime

# Crear listas para guardar los datos
marcas = []
gas95s = []
gas98s = []
gasoils = []
marcas = []
direcciones = []
poblaciones = []
horarios = []

# Definimos la funcion que busca y recupera los enlaces de las gasolineras de Barcelona
def get_comunity_links(url):
    comu_links = []
    provi_links = []
    try:
        page = requests.get(url)
    except HTTPError as e:
        print(e)
    except URLError:
        print("Server down or incorrect domain")
    else:
        res = BeautifulSoup(page.content,"html.parser")
        for link in res.find_all('a'):
            if link.has_attr('href'):
                if '/c/precio-gasoil-calefaccion-' in link.attrs['href']:
                    comu_links.append(link.attrs['href'])
                if '/p/precio-gasoil-calefaccion-' in link.attrs['href']:
                    provi_links.append(link.attrs['href'])
    return comu_links, provi_links

def get_provincia_links(comu_links,provi_links):
    #provi_links = []
    for comu_link in comu_links:
        try:
            html = ("https://www.clickgasoil.com" + comu_link)
            page = requests.get(html)
        except HTTPError as e:
            print(e)
        except URLError:
            print("Server down or incorrect domain")
        else:
            #res = BeautifulSoup(open(page, 'r'),"html.parser",from_encoding="iso-8859-1")
            res = BeautifulSoup(page.content,"html.parser", from_encoding="latin1")
        for link in res.find_all('a'):
            if link.has_attr('href'):
                if '/p/precio-gasoil-calefaccion-' in link.attrs['href']:
                    provi_links.append(link.attrs['href'])
                if '/p/precio-de-gasoil-calefaccion-' in link.attrs['href']:
                    provi_links.append(link.attrs['href'])
    return provi_links

def get_city_links(provi_links):
    city_links = []
    for provi_link in provi_links:
        try:
            html = ("https://www.clickgasoil.com" + provi_link)
            page = requests.get(html)
        except HTTPError as e:
            print(e)
        except URLError:
            print("Server down or incorrect domain")
        else:
            #soup = BeautifulSoup(open(page, 'r'),"html.parser",from_encoding="iso-8859-1")
            soup = BeautifulSoup(page.content,"html.parser", from_encoding="latin1")
        for link in soup.find_all('a'):
            if link.has_attr('href'):
                if '/m/precio-gasoil-calefaccion-' in link.attrs['href']:
                    city_links.append(link.attrs['href'])
                if '/m/precio-de-gasoil-calefaccion-' in link.attrs['href']:
                    city_links.append(link.attrs['href'])
    return city_links

def get_station_links(city_links):
    station_links = []
    for city_link in city_links:
        try:
            html = ("https://www.clickgasoil.com" + city_link)
            page = requests.get(html)
        except HTTPError as e:
            print(e)
        except URLError:
            print("Server down or incorrect domain")
        else:
            # Estructuramos el contenido de la web obtenido para ser consultado facilmente
            #estacion = BeautifulSoup(open(html, 'r'),"html.parser",from_encoding="iso-8859-1")
            estacion = BeautifulSoup(page.content,"html.parser")

            #Buscamos los links:
            for link in estacion.find_all('a'):
                if link.has_attr('href'):
                    if '/g/' in link.attrs['href']:
                        station_links.append(link.attrs['href'])
    return station_links

# Definimos la funcion que busca y recupera todos los datos referente a cada gasolinera
def get_info(station):
    try:
        html = ("https://www.clickgasoil.com" + station)
        headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.86 Safari/537.36"}
        resp = requests.get(html, headers=headers)
        http_encoding = resp.encoding if 'charset' in resp.headers.get('content-type', '').lower() else None
        html_encoding = EncodingDetector.find_declared_encoding(resp.content, is_html=True)
        encoding = html_encoding or http_encoding
        #page = requests.get(html)
    except HTTPError as e:
        print(e)
    except URLError:
        print("Server down or incorrect domain")
    else:
        # Estructuramos el contenido de la web obtenido para ser consultado facilmente
        #soup = BeautifulSoup.BeautifulSoup(content.decode('utf-8','ignore'))
        #soup = BeautifulSoup(page.content,"html.parser", from_encoding="latin1")
        soup = BeautifulSoup(resp.content, 'lxml', from_encoding=encoding)
        
        # Buscamos la información general de cada gasolinera: Marca, Direcion, Poblacion, Horario
        info_gasolinera = soup.find("div", {"class":"column small-4 datos_gasolinera"})
        
        # Buscamos ahora las etiquetas div que hemos detectado, son las que engloban la información que buscamos
        div_lista = info_gasolinera.find_all('div')
        
        # Recorremos la información y guardamos los datos que nos interesan
        for i,dato in enumerate(div_lista):
            temp = dato.text.split(":")
            if (i==0): 
                marca = temp[1]
            if (i==1):
                direccion = temp[1]
            if (i==2):
                poblacion = temp[1]
            if (i==3):
                horario = temp[1] + temp[2]
                
        # Buscamos la tabla que contiene los precios de los combustibles a fecha de la consulta
        table = soup.find("table", { "class" : "small-12" })
        
        # Buscamos todas las filas de esta tabla y las guardamos en una lista
        td_list = table.find_all('td')
        
        # Recorremos la información y guardamos los datos que nos interesan
        for i,element in enumerate(td_list):
            if(i == 1):
                precio_gasoil = td_list[1].text[:-1]
                gasoil = precio_gasoil
            if (i == 7):
                precio_gas95 = td_list[7].text[:-1]
                gas95 = precio_gas95
            if (i == 9):
                precio_gas98 = td_list[9].text[:-1]
                gas98 = precio_gas98
            
        # Devolvemos los datos
    return marca,gasoil,gas95,gas98,direccion,poblacion,horario

def precios_gasolineras(station_links):
    # Para cada estacion, vamos a su pagina donde obtenemos todos sus datos y los precios del combustible del dia
    for station in station_links:
        marca,gasoil,gas95,gas98,direccion,poblacion,horario = get_info(station)

        # Guardamos los datos en listas que nos permitiran luego crear un dataframe que exportaremos.
        marcas.append(marca)
        gasoils.append(gasoil)
        gas95s.append(gas95)
        gas98s.append(gas98)
        direcciones.append(direccion)
        poblaciones.append(poblacion)
        horarios.append(horario)

    # Creamos un dataframe con todos los datos obtenidos
    df = pd.DataFrame(
        {'Marca':marcas,
         'Gasolina 95':gas95s,
         'Gasolina 98':gas98s,
         'Gasóleo A':gasoils,
         'Direccion':direcciones,
         'Poblacion':poblaciones,
         'Horario':horarios,
         'Fecha': datetime.datetime.now()})
    return df



# Parseando la web ClickGasoil que nos ofrece los precios de Gasóleo A, Gasolina 95 y Gasolina 98 de cada gasolinera de Barcelona
url = "https://www.clickgasoil.com/c/precio-gasoil-calefaccion"
comu_links, provi_links = get_comunity_links(url)
#print (len(comu_links))
#print (len(provi_links))
provi_links = get_provincia_links(comu_links,provi_links)
#print (len(provi_links)) ## 52 provincias
city_links = get_city_links(provi_links)


#provincia = 'barcelona'
#city_links_query = []

    #print (link)
    #if 'barcelona' in link:
        #city_links_query.append(city_links[i])
        
#print (len(city_links_query))
city_links2 = np.random.choice(city_links,300)
#print (city_links2)
#print (len(city_links2))
#print (city_links[1:10])
station_links = get_station_links(city_links2)
#query_links = []
station_links2 = np.random.choice(station_links,1000)
#print (query_links)


df = precios_gasolineras(station_links)
df.head(10)

# Exportamos los datos en un archivos csv
df.to_csv("gasolineras_precios_pruebas_espanya.csv", sep=',')

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHA

In [ ]:
texts = ['532525-barcelona-tes2','532525-madrid-tes2','532525-barcelona-tes2']
for i,text in enumerate(texts):
    if 'mad' in text:
        print (texts[i])

In [3]:
text = "1.589€"
x = text[:-1]
print (x)

1.589
